In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#!pip install pyswarms

In [5]:
import torch
import json
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.optim as optim
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from pyswarms.single.global_best import GlobalBestPSO
import random

In [6]:
import os

# Define the new directory path
new_directory = '/content/drive/MyDrive/Colab Notebooks'

# Change the current working directory to the new one
os.chdir(new_directory)

# Verify the change by printing the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/MyDrive/Colab Notebooks


In [7]:
# Load the datasets
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

train_data = load_dataset('Datasets/train.jsonl')
validation_data = load_dataset('Datasets/validation.jsonl')


# Limitation of size of data
train_data = random.sample(train_data, k= 4000)
validation_data = random.sample(validation_data, k= 500)


def text_extract (data):
    text, label = [], []
    for idx in range(len(data)):
        text.append(data[idx]['text'])
        label.append(data[idx]['label'])
    return text, label

train_texts, train_labels= text_extract (train_data)

dev_texts, dev_labels= text_extract (validation_data)

#test_texts, test_labels= text_extract (test_data)

# Load the pre-trained BERT tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

# Tokenize the input texts
tokenized_train_texts = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_dev_texts = tokenizer(dev_texts, padding=True, truncation=True, return_tensors='pt')

# Convert the labels to tensor
train_labels = torch.tensor(train_labels)

dev_labels = torch.tensor(dev_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

In [8]:
# Define the objective function
def objective_function(params):

    Costs = np.zeros(params.shape[0])
    for i in range(params.shape[0]):
        learning_rate= params[i, 0]
        batch_size= int(params[i, 1])
        weight_decay= params[i, 2]
        num_epochs= 3


    # Create TensorDatasets for train, dev, and test sets
    train_dataset = TensorDataset(tokenized_train_texts['input_ids'], tokenized_train_texts['attention_mask'], train_labels)

    dev_dataset = TensorDataset(tokenized_dev_texts['input_ids'], tokenized_dev_texts['attention_mask'], dev_labels)

    # Create DataLoaders for train, dev, and test sets
    train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)

    dev_dataloader = DataLoader(dev_dataset, batch_size= batch_size, shuffle= False)

    # Load the pre-trained BERT model for sequence classification
    #model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels= 6)
    model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=6)

    # Freeze BERT parameters
    for param in model.base_model.parameters():
        param.requires_grad = False

    # Define the optimizer for training the softmax layer
    optimizer = optim.Adam(model.classifier.parameters(), lr= learning_rate, weight_decay= weight_decay)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    # Define the loss function
    criterion = nn.CrossEntropyLoss()


    # Train the model
    best_dev_accuracy = 0.0
    best_model_state_dict = None
    Validation_results= []

    for epoch in range(num_epochs):
        # Training loop
        model.train()
        for batch in train_dataloader:
            input_ids, attention_mask, batch_labels = batch

            input_ids= input_ids.to(device)
            attention_mask= attention_mask.to(device)
            batch_labels= batch_labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids= input_ids, attention_mask= attention_mask, labels= batch_labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

        # Validation loop
        model.eval()
        dev_correct = 0
        total_dev = 0
        y_true = []
        y_pred = []
        loss_epoch= []
        with torch.no_grad():
            for batch in dev_dataloader:
                input_ids, attention_mask, batch_labels = batch

                input_ids= input_ids.to(device)
                attention_mask= attention_mask.to(device)
                batch_labels= batch_labels.to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                _, predicted = torch.max(logits, 1)

                # Append true labels and predicted labels for later use
                y_true.extend(batch_labels.tolist())
                y_pred.extend(predicted.tolist())

                # Calculate the loss
                loss = criterion(logits, batch_labels)
                loss_epoch.append(loss)

        # Calculate accuracy and F1 score
        f1 = f1_score(y_true, y_pred, average='weighted')
        accuracy = accuracy_score(y_true, y_pred)

        # Calculate the average loss
        loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
        average_loss= np.mean(loss_epoch_np)
        print(f'learning_rate: {learning_rate}, batch_size: {batch_size}, weight_decay: {weight_decay}, num_epochs: {num_epochs}')
        print(f'epoch No. : {epoch}, Devset Accuracy : {round(accuracy,5)}, Devset f1_score : {round(f1,5)}, Average loss: {round(average_loss.tolist(),5)}')
        print()
    validation_loss= average_loss

    Costs[i]= (1/f1)**2

    # Return the validation loss as the objective value
    return Costs

In [9]:
# Define the bounds for the hyperparameters
lower_bound = np.array([1e-5, 16, 1e-5])
upper_bound = np.array([1e-2, 256, 1e-3])
bounds = (lower_bound, upper_bound)

In [10]:
# Initialize the optimizer
options={'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = GlobalBestPSO(n_particles= 6, dimensions= 3, options=options, bounds=bounds)

# Run the optimization
best_costs, best_hyperparams = optimizer.optimize(objective_function, iters= 20)

# Print the best hyperparameters found
print("Best position:", best_hyperparams)
print("Best cost:", best_costs)

2024-02-05 14:01:53,787 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/20

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


learning_rate: 0.0016997161546032083, batch_size: 190, weight_decay: 0.00029402594132142354, num_epochs: 3
epoch No. : 0, Devset Accuracy : 0.364, Devset f1_score : 0.27563, Average loss: 1.60002

learning_rate: 0.0016997161546032083, batch_size: 190, weight_decay: 0.00029402594132142354, num_epochs: 3
epoch No. : 1, Devset Accuracy : 0.394, Devset f1_score : 0.27864, Average loss: 1.57359



pyswarms.single.global_best:   5%|▌         |1/20, best_cost=0

...

learning_rate: 0.009755231580132239, batch_size: 218, weight_decay: 0.0006931419888689351, num_epochs: 3
epoch No. : 2, Devset Accuracy : 0.38, Devset f1_score : 0.28673, Average loss: 1.58275



Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


learning_rate: 0.00934195625647743, batch_size: 218, weight_decay: 0.0004690122200195672, num_epochs: 3
epoch No. : 0, Devset Accuracy : 0.37, Devset f1_score : 0.21931, Average loss: 1.63837

learning_rate: 0.00934195625647743, batch_size: 218, weight_decay: 0.0004690122200195672, num_epochs: 3
epoch No. : 1, Devset Accuracy : 0.376, Devset f1_score : 0.27905, Average loss: 1.6079



pyswarms.single.global_best: 100%|██████████|20/20, best_cost=0
2024-02-05 14:20:54,632 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [8.17743188e-03 2.18181349e+02 2.54979231e-04]


learning_rate: 0.00934195625647743, batch_size: 218, weight_decay: 0.0004690122200195672, num_epochs: 3
epoch No. : 2, Devset Accuracy : 0.392, Devset f1_score : 0.29636, Average loss: 1.58194

Best position: [8.17743188e-03 2.18181349e+02 2.54979231e-04]
Best cost: 0.0


In [11]:
learning_rate= best_hyperparams[0]
batch_size= int(best_hyperparams[1])
weight_decay= best_hyperparams[2]

learning_rate, batch_size, weight_decay

(0.008177431877937414, 218, 0.00025497923120386347)